# Predicting risks of toxic algal blooms in California coastal region

## Data:
1. Southern California Coastal Ocean Observing System (SCCOOS) harmful algae tracking data downloaded from http://www.sccoos.org/query/.
2. Pacific Fisheries Environmental Laboratory ocean upwelling index downloaded from https://www.pfeg.noaa.gov/products/PFEL/modeled/indices/upwelling/NA/data_download.html. The daily data for two stations (36N122W and 39N119W) were downloaded.
3. National Oceanic and Atmospheric Administration Oceanic Niño Index (ONI) downloaded from http://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php.<br>
##### All data downloaded in May 2018.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Data Wrangling

In [2]:
# Read data
sccoos_data = pd.read_csv("data/raw/SCCOOS_data.csv", skiprows=7)
sccoos_data.head()

,year,month,day,time,latitude,longitude,depth (m),location,Akashiwo sanguinea (cells/L),Alexandrium spp. (cells/L),...,Phaeophytin 1 (mg/m3),Phaeophytin 2 (mg/m3),Phosphate (uM),Prorocentrum spp. (cells/L),Pseudo-nitzschia delicatissima group (cells/L),Pseudo-nitzschia seriata group (cells/L),Silicate (uM),Volume Settled for counting (mL),Water Temperature (C),Volume for counting (mL)
0,2008,6,30,15:00:00,32.8670,-117.2570,0.0,Scripps Pier,0.0,0.0,...,0.59,0.61,0.240,4640.0,8560.0,480.0,5.640,50.0,19.8,NaN
1,2008,6,30,16:20:00,33.6061,-117.9311,0.0,Newport Pier,0.0,0.0,...,0.90,1.20,0.330,10399.0,5200.0,0.0,3.900,25.0,18.5,NaN
2,2008,6,30,16:30:00,34.4080,-119.6850,0.0,Stearns Wharf,0.0,0.0,...,2.05,NaN,0.851,470000.0,55000.0,345000.0,6.932,50.0,18.0,NaN
3,2008,7,2,14:50:00,32.8670,-117.2570,0.0,Scripps Pier,NaN,NaN,...,0.53,0.52,0.190,NaN,NaN,NaN,6.500,NaN,19.7,NaN
4,2008,7,7,14:35:00,34.0080,-118.4990,0.0,Santa Monica Pier,0.0,1122.0,...,NaN,NaN,0.290,9724.0,2244.0,0.0,6.710,NaN,21.0,500.0


In [3]:
# Consolidate year,month,day into one column
sccoos_data['date'] = pd.to_datetime(sccoos_data[['year', 'month', 'day']]).dt.date

In [4]:
print('We have {} entries, {} of which have no domoic acid data.'\
      .format(len(sccoos_data), sum(sccoos_data['Domoic Acid (ng/mL)'].isna())))

We have 3490 entries, 2342 of which have no domoic acid data.


#### The key data here (Domoic Acid) has a lot of missing data. After communication with Dr. Jayme Smith, who collected data, I found out a lot of the data that were below detection (essentially zero) were mislabelled as NaN. I then collected domoic acid data from her, and used those to correct the mistakes here. Her data were in Excel files, but separated for different sampling sites.

In [5]:
# Read one Excel file
data = pd.read_excel('data/raw/Cal Poly DA Results.xls')
data.head()

,Sample ID,Vol Filt,DA (ng/mL),Date Sent,Sent To,Unnamed: 5
0,CP080815,250,bd,2010-05-11,irobbins@calpoly.edu; samcrankin@yahoo.com; an...,NaN
1,CP080819,265,bd,2009-01-22,irobbins@calpoly.edu; samcrankin@yahoo.com; an...,NaN
2,CP080826,250,bd,2009-01-22,irobbins@calpoly.edu; samcrankin@yahoo.com; an...,NaN
3,CP080902,250,bd,2009-01-22,irobbins@calpoly.edu; samcrankin@yahoo.com; an...,NaN
4,CP080908,175,bd,2009-05-26,irobbins@calpoly.edu; samcrankin@yahoo.com; an...,NaN


In [6]:
# Parse date from sample ID, add location name, and discard irrelevant columns.
data = data[['Sample ID', 'DA (ng/mL)']]
data['location'] = 'Cal Poly Pier'
data['Sample ID'] = data['Sample ID'].str.replace('CP\s*', '')
data['date'] = pd.to_datetime(data['Sample ID'].astype(str), yearfirst=True).dt.date
da_data = data.drop('Sample ID', axis = 1)
da_data.head()

,DA (ng/mL),location,date
0,bd,Cal Poly Pier,2008-08-15
1,bd,Cal Poly Pier,2008-08-19
2,bd,Cal Poly Pier,2008-08-26
3,bd,Cal Poly Pier,2008-09-02
4,bd,Cal Poly Pier,2008-09-08


In [7]:
# Process other Excel files in similar fashion, and concatenate them together
# Each Excel files look different though, slightly modify code for different formats
# Excel file #2
data = pd.read_excel('data/raw/Goleta Pier DA Results.xlsx', header=None)
data.head()

,0,1,2,3,4,5
0,Goleta Pier,090421,200,0.54,2009-05-26,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...
1,Goleta Pier,090427,200,0.46,2009-05-26,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...
2,Goleta Pier,090504,200,2.64,2009-05-26,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...
3,Goleta Pier,090511,200,0.72,2009-05-26,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...
4,Goleta Pier,090518,200,0.25,2009-07-16,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...


In [8]:
data = data.iloc[:, [0,1,3]]
data.columns = ['location', 'date', 'DA (ng/mL)']
data['date'] = pd.to_datetime(data['date'], yearfirst=True).dt.date
da_data = pd.concat([da_data, data], ignore_index=True)

In [9]:
# Excel file #3
data = pd.read_excel('data/raw/New Port DA Results.xls')
data.head()

,SampleID,Location Code,Date Collected,Time Collected (PST),Temp (°C),Chl Volume Filtered (mL),Chl1 (mg/m3),Chl2 (mg/m3),Avg Chloro (mg/m3),Phaeo1 (mg/m3),...,DA Volume Filtered (mL),Domoic Acid (ng/mL),Volume Settled for counting (mL),Akashiwo sanguinea (cells/L),Alexandrium spp. (cells/L),Dinophysis spp. (cells/L),Lingulodinium polyedrum (cells/L),Prorocentrum spp. (cells/L),Pseudo-nitzschia delicatissima group (cells/L),Pseudo-nitzschia seriata group (cells/L)
0,NP063008,NP,2008-06-30,09:20:00,18.5,100,2.50,2.50,2.500,0.9,...,200,nd,25.0,0.0,0.0,0.000000,0.000000,10399.375038,5199.687519,0.000000
1,NP070808,NP,2008-07-08,06:50:00,NaN,100,2.04,2.13,2.085,1.0,...,200,nd,25.0,0.0,0.0,0.000000,0.000000,0.000000,363978.126315,41597.500150
2,NP071408,NP,2008-07-14,08:05:00,19.5,100,5.90,7.21,6.555,3.0,...,200,nd,25.0,0.0,0.0,5199.687519,5199.687519,57196.562707,41597.500150,20798.750075
3,NP072108,NP,2008-07-21,07:24:00,20.0,100,1.44,1.38,1.410,0.6,...,200,nd,25.0,0.0,0.0,0.000000,0.000000,5199.687519,0.000000,0.000000
4,NP072808,NP,2008-07-28,07:45:00,22.0,100,1.57,1.71,1.640,0.6,...,200,nd,25.0,0.0,0.0,0.000000,0.000000,0.000000,41597.500150,0.000000


In [10]:
data = data[['Date Collected', 'Domoic Acid (ng/mL)']]
data.columns = ['date', 'DA (ng/mL)']
data['location'] = 'Newport Pier'
data['date'] = data['date'].dt.date
da_data = pd.concat([da_data, data], ignore_index=True)

In [11]:
# Excel file #4
data = pd.read_excel('data/raw/Scripps Pier DA Results.xls')
data.head()

,Location,Sample ID,Vol Filt,DA (ng/mL),Date Sent,Sent To,Unnamed: 6
0,SIO,20080630,200,bd,2009-01-22,mlcarter@ucsd.edu,NaN
1,SIO,20080707,200,bd,2009-02-03,mlcarter@ucsd.edu; hilbern@coast.ucsd.edu,NaN
2,SIO,20080714,200,bd,2009-01-22,mlcarter@ucsd.edu,NaN
3,SIO,20080721,200,bd,2009-01-22,mlcarter@ucsd.edu,NaN
4,SIO,20080728,200,bd,2009-02-03,mlcarter@ucsd.edu; hilbern@coast.ucsd.edu,NaN


In [12]:
data = data[['Sample ID', 'DA (ng/mL)']]
data['location'] = 'Scripps Pier'
data['date'] = pd.to_datetime(data['Sample ID'].astype(str),yearfirst=True).dt.date
data = data.drop('Sample ID', axis = 1)
da_data = pd.concat([da_data, data], ignore_index=True)

In [13]:
# Excel file #5
data = pd.read_excel('data/raw/Stern Wharf DA Results.xls')
data.head()

,Location,Sample ID,Vol Filt,DA (ng/mL),Date Sent,Sent To,Unnamed: 6,Location.1,Sample ID.1,Vol Filt.1,DA (ng/mL).1,Date Sent.1,Sent To.1,Unnamed: 13,Location.2,Sample ID.2,Vol Filt.2,DA (ng/mL).2,Date Sent.2,Sent To.2
0,Stearn's Wharf,080630,200,0.05,2009-01-22,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stearn's Wharf,080707,200,0.05,2009-01-22,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Stearn's Wharf,080714,200,bd,2009-01-22,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stearn's Wharf,080722,200,bd,2009-01-22,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Stearn's Wharf,080728,200,bd,2009-01-22,brzezins@lifesci.ucsb.edu; goodman@lifesci.ucs...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data = data[['Sample ID', 'DA (ng/mL)']]
data['location'] = 'Stearns Wharf'
data['date'] = pd.to_datetime(data['Sample ID'].astype(str),yearfirst=True).dt.date
data = data.drop('Sample ID', axis = 1)
da_data = pd.concat([da_data, data], ignore_index=True)

In [15]:
# Excel file #6
data = pd.read_excel('data/raw/Santa Monica DA Results.xlsx')
data.head()

,Location,Date,Sample ID,Vol Filt,DA (ng/mL),Date Sent,Sent To
0,Santa Monica Pier,2008-10-20 00:00:00,1,200,0.04,12209,rshipe@gmail.com
1,Santa Monica Pier,2008-10-28 00:00:00,2,200,bd,12209,rshipe@gmail.com
2,Santa Monica Pier,2008-11-03 00:00:00,3,200,bd,12209,rshipe@gmail.com
3,Santa Monica Pier,2008-11-17 00:00:00,NaN,200,bd,12209,rshipe@gmail.com
4,Santa Monica Pier,2008-11-24 00:00:00,5,200,0.04,12209,rshipe@gmail.com


In [16]:
data = data[['Date', 'DA (ng/mL)']]
data['location'] = 'Santa Monica Pier'
data['date'] = pd.to_datetime(data['Date'].astype(str)).dt.date
data = data.drop('Date', axis = 1)
da_data = pd.concat([da_data, data], ignore_index=True)

In [17]:
# Drop duplicates
da_data = da_data.drop_duplicates(subset=['location', 'date'])
# Change 'bd' and 'nd' which are short for 'below detection' and 'not detected' to 0
da_data['DA (ng/mL)'].replace(['bd','nd'], 0, inplace=True)
# Merge with the SCCOOS data table
all_data = pd.merge(sccoos_data, da_data, on=['location', 'date'], how='left')
all_data.head()

,year,month,day,time,latitude,longitude,depth (m),location,Akashiwo sanguinea (cells/L),Alexandrium spp. (cells/L),...,Phosphate (uM),Prorocentrum spp. (cells/L),Pseudo-nitzschia delicatissima group (cells/L),Pseudo-nitzschia seriata group (cells/L),Silicate (uM),Volume Settled for counting (mL),Water Temperature (C),Volume for counting (mL),date,DA (ng/mL)
0,2008,6,30,15:00:00,32.8670,-117.2570,0.0,Scripps Pier,0.0,0.0,...,0.240,4640.0,8560.0,480.0,5.640,50.0,19.8,NaN,2008-06-30,0.00
1,2008,6,30,16:20:00,33.6061,-117.9311,0.0,Newport Pier,0.0,0.0,...,0.330,10399.0,5200.0,0.0,3.900,25.0,18.5,NaN,2008-06-30,0.00
2,2008,6,30,16:30:00,34.4080,-119.6850,0.0,Stearns Wharf,0.0,0.0,...,0.851,470000.0,55000.0,345000.0,6.932,50.0,18.0,NaN,2008-06-30,0.05
3,2008,7,2,14:50:00,32.8670,-117.2570,0.0,Scripps Pier,NaN,NaN,...,0.190,NaN,NaN,NaN,6.500,NaN,19.7,NaN,2008-07-02,NaN
4,2008,7,7,14:35:00,34.0080,-118.4990,0.0,Santa Monica Pier,0.0,1122.0,...,0.290,9724.0,2244.0,0.0,6.710,NaN,21.0,500.0,2008-07-07,NaN


In [18]:
# Replace Domoic Acid values with correct data if the original data is NaN.
all_data['Domoic Acid (ng/mL)'].fillna(all_data['DA (ng/mL)'], inplace=True)
all_data = all_data.drop('DA (ng/mL)', axis=1)
print('We have {} entries, {} of which have no domoic acid data.'.\
      format(len(all_data), sum(all_data['Domoic Acid (ng/mL)'].isna())))

We have 3490 entries, 740 of which have no domoic acid data.


#### The problem of missing domoic acid data is much better now.
### Get the ocean upwelling index data

In [19]:
# There are only two stations that are relevant to our study, 36N122W and 33N119W
url = 'https://www.pfeg.noaa.gov/products/PFELData/upwell/daily/p10dayac.all'
n36_data = pd.read_csv(url, skiprows=5, delim_whitespace=True)
# According to website, missing data are marked by -9999
n36_data['Index'] = n36_data['Index'].replace(-9999, np.nan)
n36_data.head()

,YYYYMMDD,Index
0,19670101,74.0
1,19670102,17.0
2,19670103,-24.0
3,19670104,-27.0
4,19670105,112.0


In [20]:
n36_data.columns = ['date', 'Upwelling Index 36']
n36_data['date'] = pd.to_datetime(n36_data['date'].astype(str),yearfirst=True).dt.date
n36_data.head()

,date,Upwelling Index 36
0,1967-01-01,74.0
1,1967-01-02,17.0
2,1967-01-03,-24.0
3,1967-01-04,-27.0
4,1967-01-05,112.0


In [21]:
# Do the same for station 33N119W
url = 'https://www.pfeg.noaa.gov/products/PFELData/upwell/daily/p11dayac.all'
n33_data = pd.read_csv(url, skiprows=5, delim_whitespace=True)
n33_data['Index'] = n33_data['Index'].replace(-9999, np.nan)
n33_data.columns = ['date', 'Upwelling Index 33']
n33_data['date'] = pd.to_datetime(n33_data['date'].astype(str),yearfirst=True).dt.date

In [22]:
# Merge the two indices with the main dataset
all_data = pd.merge(all_data, n36_data, on='date', how='left')
all_data = pd.merge(all_data, n33_data, on='date', how='left')
# Look at where the sampling locations are
all_data[['location', 'latitude', 'longitude']].drop_duplicates()

,location,latitude,longitude
0,Scripps Pier,32.867000,-117.257000
1,Newport Pier,33.606100,-117.931100
2,Stearns Wharf,34.408000,-119.685000
4,Santa Monica Pier,34.008000,-118.499000
34,Cal Poly Pier,35.170000,-120.741000
271,Goleta Pier,34.416108,-119.828749
1169,Santa Cruz Wharf,36.958000,-122.017000
1229,Monterey Wharf,36.603686,-121.889271


In [23]:
# For stations south of N34, I will use N33 data. For stations north of N35, I will use N36 data.
# For all others between N34 and N35, I will use the average of N33 and N36.
df = all_data[['latitude', 'Upwelling Index 33', 'Upwelling Index 36']]
upwelling = list()
for index, row in df.iterrows():
    if row['latitude'] < 34:
        upwelling.append(row['Upwelling Index 33'])
    elif row['latitude'] > 35:
        upwelling.append(row['Upwelling Index 36'])
    else:
        upwelling.append((row['Upwelling Index 33']+row['Upwelling Index 36']) / 2)
all_data['Upwelling Index'] = pd.Series(upwelling)
all_data = all_data.drop(['Upwelling Index 36', 'Upwelling Index 33'], axis=1)

### Get the Oceanic Niño Index (ONI) data

In [24]:
# ONI data are in 3 month rolling averages. 
oni_data = pd.read_excel('data/raw/ONI_data_2000.xlsx')
oni_data.head()

,Year,DJF,JFM,FMA,MAM,AMJ,MJJ,JJA,JAS,ASO,SON,OND,NDJ
0,2000,-1.7,-1.4,-1.1,-0.8,-0.7,-0.6,-0.6,-0.5,-0.5,-0.6,-0.7,-0.7
1,2001,-0.7,-0.5,-0.4,-0.3,-0.3,-0.1,-0.1,-0.1,-0.2,-0.3,-0.3,-0.3
2,2002,-0.1,0.0,0.1,0.2,0.4,0.7,0.8,0.9,1.0,1.2,1.3,1.1
3,2003,0.9,0.6,0.4,0.0,-0.3,-0.2,0.1,0.2,0.3,0.3,0.4,0.4
4,2004,0.4,0.3,0.2,0.2,0.2,0.3,0.5,0.6,0.7,0.7,0.7,0.7


In [25]:
# To apply the data to single month, I simply use the 3 month period that contain the target month in the middle. 
# For example, JFM(Jan, Feb, Mar) will be used for Feb.
month_conversion = dict()
for index, value in enumerate(oni_data.columns):
    month_conversion[value] = index
oni_data = oni_data.melt(id_vars='Year', var_name='3M', value_name='ONI')
oni_data['month'] = oni_data['3M'].map(month_conversion)
oni_data = oni_data.drop('3M', axis = 1).rename(columns = {'Year':'year'})
oni_data.head()

,year,ONI,month
0,2000,-1.7,1
1,2001,-0.7,1
2,2002,-0.1,1
3,2003,0.9,1
4,2004,0.4,1


In [26]:
# Merge with main dataset
all_data = pd.merge(all_data, oni_data, on=['year', 'month'], how='left')
all_data.head()

,year,month,day,time,latitude,longitude,depth (m),location,Akashiwo sanguinea (cells/L),Alexandrium spp. (cells/L),...,Prorocentrum spp. (cells/L),Pseudo-nitzschia delicatissima group (cells/L),Pseudo-nitzschia seriata group (cells/L),Silicate (uM),Volume Settled for counting (mL),Water Temperature (C),Volume for counting (mL),date,Upwelling Index,ONI
0,2008,6,30,15:00:00,32.8670,-117.2570,0.0,Scripps Pier,0.0,0.0,...,4640.0,8560.0,480.0,5.640,50.0,19.8,NaN,2008-06-30,164.0,-0.5
1,2008,6,30,16:20:00,33.6061,-117.9311,0.0,Newport Pier,0.0,0.0,...,10399.0,5200.0,0.0,3.900,25.0,18.5,NaN,2008-06-30,164.0,-0.5
2,2008,6,30,16:30:00,34.4080,-119.6850,0.0,Stearns Wharf,0.0,0.0,...,470000.0,55000.0,345000.0,6.932,50.0,18.0,NaN,2008-06-30,112.5,-0.5
3,2008,7,2,14:50:00,32.8670,-117.2570,0.0,Scripps Pier,NaN,NaN,...,NaN,NaN,NaN,6.500,NaN,19.7,NaN,2008-07-02,137.0,-0.4
4,2008,7,7,14:35:00,34.0080,-118.4990,0.0,Santa Monica Pier,0.0,1122.0,...,9724.0,2244.0,0.0,6.710,NaN,21.0,500.0,2008-07-07,38.5,-0.4


## Clean up the data
1. Year,month,day columns contain information already in date column. No need to keep them.
2. Latitude, logitude, location will be removed, because the model is intended for the coast region, not just these stations.
3. Depth is also not relevant because almost all of them are 0.
4. 'DA Volume Filtered', 'Volume Settled for counting', and 'Volume for counting' are indicative of detection limit, not relevant here, should be removed.

In [44]:
# Remove rows where the target data (domonic acid) is missing
clean_data = all_data.dropna(subset=['Domoic Acid (ng/mL)'])
# Remove all the above columns
clean_data = clean_data.drop(['year','day','time','latitude','longitude',
                              'depth (m)','location','DA Volume Filtered (mL)',
                              'Volume Settled for counting (mL)',
                              'Volume for counting (mL)'], axis=1)
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2750 entries, 0 to 3466
Data columns (total 26 columns):
month                                             2750 non-null int64
Akashiwo sanguinea (cells/L)                      2368 non-null float64
Alexandrium spp. (cells/L)                        2628 non-null float64
Ammonia (uM)                                      2564 non-null float64
Chlorophyll (mg/m3)                               2482 non-null float64
Chlorophyll 1 (mg/m3)                             2441 non-null float64
Chlorophyll 2 (mg/m3)                             1643 non-null float64
Dinophysis spp. (cells/L)                         2550 non-null float64
Domoic Acid (ng/mL)                               2750 non-null float64
Lingulodinium polyedrum (cells/L)                 2368 non-null float64
Nitrate (uM)                                      2335 non-null float64
Nitrite (uM)                                      1915 non-null float64
Other Diatoms (cells/L)        

In [45]:
# 'Other Diatoms', 'Other Dinoflagellates', 'Phaeophytin', 'Phaeopythin 1', 'Phaeophytin 2' have too many missing data. 
# It's best to drop these columns.
clean_data = clean_data.drop(['Other Diatoms (cells/L)', 'Other Dinoflagellates (cells/L)',
                              'Phaeophytin (mg/m3)', 'Phaeophytin 1 (mg/m3)',
                              'Phaeophytin 2 (mg/m3)'], axis=1)
# Let's try to fill in some missing data here.
# 'Chlorophyll 1' and 'Chlorophyll 2' are the two replicates of chlorophyll measurement.
# Calculate the mean to fill in NaN in 'Chlorophyll', then remove the two replicates.
chl = clean_data[['Chlorophyll 1 (mg/m3)', 'Chlorophyll 2 (mg/m3)']].mean(axis=1)
clean_data['Chlorophyll (mg/m3)'] = clean_data['Chlorophyll (mg/m3)'].fillna(chl)
clean_data = clean_data.drop(['Chlorophyll 1 (mg/m3)', 'Chlorophyll 2 (mg/m3)'], axis=1)

In [46]:
clean_data.describe()

,month,Akashiwo sanguinea (cells/L),Alexandrium spp. (cells/L),Ammonia (uM),Chlorophyll (mg/m3),Dinophysis spp. (cells/L),Domoic Acid (ng/mL),Lingulodinium polyedrum (cells/L),Nitrate (uM),Nitrite (uM),Phosphate (uM),Prorocentrum spp. (cells/L),Pseudo-nitzschia delicatissima group (cells/L),Pseudo-nitzschia seriata group (cells/L),Silicate (uM),Water Temperature (C),Upwelling Index,ONI
count,2750.000000,2368.000000,2628.000000,2564.000000,2495.000000,2550.000000,2750.000000,2.368000e+03,2335.000000,1915.000000,2559.000000,2.368000e+03,2.368000e+03,2.629000e+03,2454.000000,2626.000000,2749.000000,2750.000000
mean,6.546909,852.271056,693.265856,6.131658,4.302069,512.375203,0.122078,7.074956e+03,2.612440,0.381265,0.738717,1.233938e+04,2.251564e+04,1.557518e+04,8.116095,16.301889,80.824118,0.018255
std,3.439306,6292.299490,8232.628294,20.421455,7.079582,2162.189580,0.729204,5.890398e+04,4.548807,1.480891,2.675740,5.111755e+04,1.502316e+05,8.032173e+04,12.259184,2.982468,79.237212,0.934128
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-0.010000,-0.020000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,-152.000000,-1.700000
25%,4.000000,0.000000,0.000000,1.040000,1.310000,0.000000,0.000000,0.000000e+00,0.344565,0.066979,0.270000,0.000000e+00,0.000000e+00,0.000000e+00,2.700000,14.200000,24.000000,-0.600000
50%,7.000000,0.000000,0.000000,2.300000,2.286360,0.000000,0.000000,0.000000e+00,0.827815,0.140000,0.410000,1.624000e+03,1.053000e+03,0.000000e+00,4.458655,16.000000,68.000000,-0.200000
75%,10.000000,0.000000,0.000000,4.634750,4.520000,187.000000,0.000000,3.740000e+02,2.855000,0.296368,0.699500,7.800000e+03,9.746000e+03,3.600000e+03,8.483750,18.000000,126.000000,0.500000
max,12.000000,126400.000000,367823.000000,530.000000,137.550000,52775.000000,16.220000,1.490400e+06,87.510000,28.702300,92.600000,1.253300e+06,5.858320e+06,2.350970e+06,157.000000,25.500000,484.000000,2.600000


In [47]:
# There are at least three outliers that need to be corrected.
# Nitrate and Nitrite concentrations can't be negative. Change all negative numbers to missing values.
clean_data.loc[clean_data['Nitrate (uM)'] < 0, 'Nitrate (uM)'] = np.nan
clean_data.loc[clean_data['Nitrite (uM)'] < 0, 'Nitrite (uM)'] = np.nan
# Water temperature cannot be 0. Change zeros to missing values.
clean_data['Water Temperature (C)'] = clean_data['Water Temperature (C)'].replace(0, np.nan)

In [48]:
# Fill other missing values using monthly means, because some if not all data has seasonal patterns.
monthly_means = clean_data.groupby('month').mean()
for column in monthly_means:
    df = monthly_means[column].rename('mean')
    clean_data = clean_data.join(df, on='month', how='left')
    clean_data[column] = clean_data[column].fillna(clean_data['mean'])
    clean_data = clean_data.drop('mean', axis=1)
# Drop the 'month' column now that I finished using it
clean_data = clean_data.drop('month', axis=1)

In [49]:
# No missing data now.
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2750 entries, 0 to 3466
Data columns (total 18 columns):
Akashiwo sanguinea (cells/L)                      2750 non-null float64
Alexandrium spp. (cells/L)                        2750 non-null float64
Ammonia (uM)                                      2750 non-null float64
Chlorophyll (mg/m3)                               2750 non-null float64
Dinophysis spp. (cells/L)                         2750 non-null float64
Domoic Acid (ng/mL)                               2750 non-null float64
Lingulodinium polyedrum (cells/L)                 2750 non-null float64
Nitrate (uM)                                      2750 non-null float64
Nitrite (uM)                                      2750 non-null float64
Phosphate (uM)                                    2750 non-null float64
Prorocentrum spp. (cells/L)                       2750 non-null float64
Pseudo-nitzschia delicatissima group (cells/L)    2750 non-null float64
Pseudo-nitzschia seriata grou

In [ ]:
# Save the clean data
clean_data.to_csv("data/clean_data.csv", index=False)